# Fine-Tuning Transformers Models with HuggingFace Trainer
In this example we'll fine-tune [BERT](https://huggingface.co/google-bert/bert-base-cased), with the [IMBD dataset](https://huggingface.co/datasets/imdb) for a Text Classification use-case using the [Trainer class](https://huggingface.co/docs/transformers/v4.37.2/en/main_classes/trainer#transformers.Trainer).

- Setup: <b>conda_python3 kernel</b> and <b>ml.g4dn.12xlarge</b> SageMaker Classic Notebook Instances

## Setup

In [ ]:
%pip install torch

In [ ]:
import datasets
import evaluate
import transformers
from datasets import load_dataset

In [ ]:
print(1+1)

In [ ]:
train_dataset = load_dataset("imdb", split="train")
test_dataset = load_dataset("imdb", split="test")
test_subset = test_dataset.select(range(100)) # we will take a subset of the data for evaluation

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# tokenize text data
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_subset.map(tokenize_function, batched=True)

## Fine-Tuning

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=1)

In [ ]:
import numpy as np
import evaluate
metric = evaluate.load("accuracy")

# eval function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test, #using test as eval
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.predict(tokenized_test)

In [ ]:
trainer.save_model("./custom_model")

In [ ]:
loaded_model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path="custom_model/")

In [ ]:
encoding = tokenizer("I am super delighted", return_tensors="pt")
res = loaded_model(**encoding)
predicted_label_classes = res.logits.argmax(-1)
predicted_label_classes